### Introduction

In this notebook I perform raw data transformation to calculate two paerameters used for the clustering method.

In [1]:
import pandas as pd

# Raw Data Preparation

## Preparing, cleaning and modifying row data.

In [2]:
rawdata = pd.read_csv("raw_data.csv", sep=";",decimal=",")

In [25]:
rawdata.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,TotalPrice
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom,15.30
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom,22.00
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34


In [26]:
rawdata.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 406829 entries, 0 to 541908
Data columns (total 9 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    406829 non-null  object        
 1   StockCode    406829 non-null  object        
 2   Description  406829 non-null  object        
 3   Quantity     406829 non-null  int64         
 4   InvoiceDate  406829 non-null  datetime64[ns]
 5   UnitPrice    406829 non-null  float64       
 6   CustomerID   406829 non-null  float64       
 7   Country      406829 non-null  object        
 8   TotalPrice   406829 non-null  float64       
dtypes: datetime64[ns](1), float64(3), int64(1), object(4)
memory usage: 31.0+ MB


### Eliminating missing values, transforming "InvoiceDate" into datetime format and calculating "TotalPrice" for each row.


In [5]:
rawdata = rawdata.dropna(axis=0)
rawdata["InvoiceDate"] = pd.to_datetime(rawdata["InvoiceDate"],dayfirst = True)
rawdata["TotalPrice"] = rawdata["Quantity"] * rawdata["UnitPrice"]

In [6]:
rawdata.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 406829 entries, 0 to 541908
Data columns (total 9 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    406829 non-null  object        
 1   StockCode    406829 non-null  object        
 2   Description  406829 non-null  object        
 3   Quantity     406829 non-null  int64         
 4   InvoiceDate  406829 non-null  datetime64[ns]
 5   UnitPrice    406829 non-null  float64       
 6   CustomerID   406829 non-null  float64       
 7   Country      406829 non-null  object        
 8   TotalPrice   406829 non-null  float64       
dtypes: datetime64[ns](1), float64(3), int64(1), object(4)
memory usage: 31.0+ MB


In [7]:
rawdata.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,TotalPrice
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom,15.30
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom,22.00
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34


### (little) EDA

In [28]:
NCust = rawdata["CustomerID"].nunique()
NInvoice = rawdata["InvoiceNo"].nunique()
NStockCode = rawdata["StockCode"].nunique()

print(f"The raw data contains {NCust} customers, {NStockCode} items and {NInvoice} invoices.")

The raw data contains 4372 customers, 3684 items and 22190 invoices.


### Calculating mean purchase value and number of invoices for each customer.

In [29]:
invoice_aggregation = rawdata[["CustomerID","InvoiceNo","TotalPrice"]].groupby(["CustomerID","InvoiceNo"]).sum().reset_index()
invoice_aggregation["PurchNum"] = invoice_aggregation["InvoiceNo"].apply(lambda x : -1 if x[0] == "C" else 1)
invoice_aggregation

,CustomerID,InvoiceNo,TotalPrice,PurchNum
0,12346.0,541431,77183.60,1
1,12346.0,C541433,-77183.60,-1
2,12347.0,537626,711.79,1
3,12347.0,542237,475.39,1
4,12347.0,549222,636.25,1
...,...,...,...,...
22185,18283.0,579673,223.61,1
22186,18283.0,580872,208.00,1
22187,18287.0,554065,765.28,1
22188,18287.0,570715,1001.32,1


In [30]:
invoice_aggregation[invoice_aggregation["TotalPrice"]<0]

,CustomerID,InvoiceNo,TotalPrice,PurchNum
1,12346.0,C541433,-77183.60,-1
23,12352.0,C545329,-463.80,-1
24,12352.0,C545330,-376.50,-1
25,12352.0,C547388,-120.33,-1
39,12359.0,C549955,-35.80,-1
...,...,...,...,...
22159,18274.0,C577832,-175.92,-1
22161,18276.0,C577386,-6.25,-1
22162,18276.0,C577390,-6.25,-1
22164,18277.0,C542086,-12.75,-1


As we can see we have invoices which cancel purchases. They start from the letter "C". The question is what to do with it. There are two approaches: one to get rid all returned purchase and treat them as never existing, second to use those returns somehow. The second approach is more ambitious but mor difficult how to use this data to market segmentation. Since I have not got more advanced knowledge how the company treads such returns I decided to remove them. The column "PurchNum" reflects that. The sum of the column shows the number of not cancelled purchases. To calculate mean invoice we have to be more subtle and cannot use the "mean" function but to calculate it manually: first to calculate the sum and then to divide it by the PurchNum manually. We will remove "NaN" at the end.

In [31]:
finaldata = invoice_aggregation[["CustomerID","TotalPrice"]].groupby(["CustomerID"]).sum()
temp = invoice_aggregation[["CustomerID","PurchNum"]].groupby(["CustomerID"]).sum()
finaldata["TotalPrice"] = finaldata["TotalPrice"]/ temp["PurchNum"]
finaldata = finaldata.rename(columns = {"TotalPrice" : "MeanInvoice"})

In [32]:
finaldata = finaldata.join(temp)
# dodac date pierwszego zakupu i obliczyc czestotliwosc
finaldata

,MeanInvoice,PurchNum
CustomerID,,
12346.0,NaN,0
12347.0,615.714286,7
12348.0,449.310000,4
12349.0,1757.550000,1
12350.0,334.400000,1
...,...,...
18280.0,180.600000,1
18281.0,80.820000,1
18282.0,176.600000,1


## Calculating the frequency of purchase

In this point we calculate the lastday - which I tread as "today" for our analysis and for each customer I calculate their first purchase and later calculate the time period the person is the customer. I have add one day to the result to round everything up to avoid zeros.

In [33]:
import datetime

In [34]:
lastday = rawdata["InvoiceDate"].max()
lastday

Timestamp('2011-12-09 12:50:00')

In [35]:
dates = rawdata[["CustomerID","InvoiceDate"]].groupby(["CustomerID"]).min()
dates["PurchasePeriod"] = dates["InvoiceDate"].apply(lambda x : pd.Timedelta(lastday-x).days+1)
dates

,InvoiceDate,PurchasePeriod
CustomerID,,
12346.0,2011-01-18 10:01:00,326
12347.0,2010-12-07 14:57:00,367
12348.0,2010-12-16 19:09:00,358
12349.0,2011-11-21 09:51:00,19
12350.0,2011-02-02 16:01:00,310
...,...,...
18280.0,2011-03-07 09:52:00,278
18281.0,2011-06-12 10:53:00,181
18282.0,2011-08-05 13:35:00,126


### Joining everything together

The lats step is to join together the two tables, remove zeros and NaN, remove temporary data and to write down the result in the binary form. This will be the ready data set for clusterisation.

In [36]:
finaldata = finaldata.join(dates[["PurchasePeriod"]])
finaldata

,MeanInvoice,PurchNum,PurchasePeriod
CustomerID,,,
12346.0,NaN,0,326
12347.0,615.714286,7,367
12348.0,449.310000,4,358
12349.0,1757.550000,1,19
12350.0,334.400000,1,310
...,...,...,...
18280.0,180.600000,1,278
18281.0,80.820000,1,181
18282.0,176.600000,1,126


In [37]:
finaldata = finaldata.dropna()
finaldata = finaldata[finaldata["PurchNum"] > 0]
finaldata["PurchFreq"] = finaldata["PurchNum"]/finaldata["PurchasePeriod"]

In [22]:
finaldata = finaldata.reset_index()
finaldata

,CustomerID,MeanInvoice,PurchNum,PurchasePeriod,PurchFreq
0,12347.0,615.714286,7,367,0.019074
1,12348.0,449.310000,4,358,0.011173
2,12349.0,1757.550000,1,19,0.052632
3,12350.0,334.400000,1,310,0.003226
4,12352.0,309.082000,5,297,0.016835
...,...,...,...,...,...
4027,18280.0,180.600000,1,278,0.003597
4028,18281.0,80.820000,1,181,0.005525
4029,18282.0,176.600000,1,126,0.007937
4030,18283.0,130.930000,16,337,0.047478


In [38]:
finaldata = finaldata.drop(["PurchNum","PurchasePeriod"],axis=1)
finaldata.head()

,MeanInvoice,PurchFreq
CustomerID,,
12347.0,615.714286,0.019074
12348.0,449.310000,0.011173
12349.0,1757.550000,0.052632
12350.0,334.400000,0.003226
12352.0,309.082000,0.016835


In [39]:
finaldata.to_pickle("data_for_KML.pkl")